## Collecting Data

To collect my Spotify Data I used this source: https://github.com/plamere/spotipy/tree/master/examples

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 


client_id = '3d5be77a0a8b4b34a19fe1c509719169'
client_secret = 'b5f6d556e9a74f48a88ec4e4a40bfc5e'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:

def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)

    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
        
    
    return ids



ids = getTrackIDs('FitnessFitnessFITNESSMINE', '695feaZvvoko76YU2NbFev')


def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  length = meta['duration_ms']
  popularity = meta['popularity']
 
  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  mode = features[0]['mode']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']

  track = [name, album, artist, length, popularity, danceability, acousticness, energy, instrumentalness, mode, liveness, loudness, speechiness, tempo, time_signature, valence]
  return track



# loop over track ids 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# create dataset
tracks_df = pd.DataFrame(tracks, columns = ['trackname', 'album', 'artist', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', ' mode', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])

In [3]:
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trackname         70 non-null     object 
 1   album             70 non-null     object 
 2   artist            70 non-null     object 
 3   length            70 non-null     int64  
 4   popularity        70 non-null     int64  
 5   danceability      70 non-null     float64
 6   acousticness      70 non-null     float64
 7   energy            70 non-null     float64
 8   instrumentalness  70 non-null     float64
 9    mode             70 non-null     int64  
 10  liveness          70 non-null     float64
 11  loudness          70 non-null     float64
 12  speechiness       70 non-null     float64
 13  tempo             70 non-null     float64
 14  time_signature    70 non-null     int64  
 15  valence           70 non-null     float64
dtypes: float64(9), int64(4), object(3)
memory usag

In [4]:
dct = sp.playlist("695feaZvvoko76YU2NbFev", fields=None, market=None, additional_types=('track', ))
result = sp.search(dct['tracks']['items'][0]['track']['artists'][0]['name'])
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

genres = artist["genres"]
artist = artist['name']
print(genres)
print (artist)

artist genres: ['dance pop', 'miami hip hop', 'pop', 'pop dance', 'pop rap']
['dance pop', 'miami hip hop', 'pop', 'pop dance', 'pop rap']
Flo Rida


In [5]:
from tqdm import tqdm
import pandas as pd

In [6]:
rows = []
for t in tqdm(range(len(tracks_df))):
    track_row = []
    
    track = dct['tracks']['items'][t]['track']['name']
    artist = dct['tracks']['items'][t]['track']['artists'][0]['name']
    
    result = sp.search(dct['tracks']['items'][t]['track']['artists'][0]['name'])
    tr = result['tracks']['items'][0]
    art = sp.artist(tr["artists"][0]["external_urls"]["spotify"])
    
    genre = art["genres"]
    try:
        track_row = [artist, genre[0]]
    except:
        track_row = [artist]
    rows.append(track_row)
    
    
genre = pd.DataFrame.from_dict(rows)

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:18<00:00,  3.75it/s]


In [7]:
genre.columns = ['artist', 'genres']

In [8]:
genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  70 non-null     object
 1   genres  67 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [9]:
df = pd.merge(tracks_df, genre,  on = 'artist') #"artist" is the common identifier
df.head()

,trackname,album,artist,length,popularity,danceability,acousticness,energy,instrumentalness,mode,liveness,loudness,speechiness,tempo,time_signature,valence,genres
0,Hola (feat. Maluma),Hola (feat. Maluma),Flo Rida,192376,57,0.757,0.02070,0.711,0.000030,1,0.1100,-5.693,0.0769,113.026,4,0.649,dance pop
1,Places,Places,Martin Solveig,202745,63,0.728,0.24800,0.854,0.001880,0,0.1490,-3.860,0.0457,121.995,4,0.620,dance pop
2,I Need You (feat. Olaf Blackwood) - Extended C...,I Need You (feat. Olaf Blackwood) [Club Mix],Armin van Buuren,330360,22,0.666,0.00151,0.872,0.137000,0,0.0933,-6.497,0.0584,127.984,4,0.255,dance pop
3,Come Get It Bae,G I R L,Pharrell Williams,201933,0,0.934,0.26600,0.798,0.000045,1,0.0962,-5.665,0.0849,120.162,4,0.895,latin
4,Let Me Love You - R3hab Remix,Let Me Love You (R3hab Remix),DJ Snake,175000,53,0.433,0.02090,0.820,0.000000,0,0.2920,-2.840,0.0924,203.968,4,0.536,dance pop


In [10]:
grouped = df.groupby(['trackname','artist'], as_index=True).size()
grouped[grouped > 1].count()

17

In [11]:
df.drop_duplicates(subset=['trackname','artist'], inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 97
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trackname         70 non-null     object 
 1   album             70 non-null     object 
 2   artist            70 non-null     object 
 3   length            70 non-null     int64  
 4   popularity        70 non-null     int64  
 5   danceability      70 non-null     float64
 6   acousticness      70 non-null     float64
 7   energy            70 non-null     float64
 8   instrumentalness  70 non-null     float64
 9    mode             70 non-null     int64  
 10  liveness          70 non-null     float64
 11  loudness          70 non-null     float64
 12  speechiness       70 non-null     float64
 13  tempo             70 non-null     float64
 14  time_signature    70 non-null     int64  
 15  valence           70 non-null     float64
 16  genres            67 non-null     object 
dtyp

In [13]:
df.to_csv("playlist3.csv", sep = ',')